# 수집한 이미지 데이터 분할 

In [23]:
import os 
os.makedirs('/Users/kibeomkim/Desktop/insta_data/train/samsung')
os.chdir('/Users/kibeomkim/Desktop/insta_data/train')
os.makedirs('/Users/kibeomkim/Desktop/insta_data/train/apple')

os.chdir('/Users/kibeomkim/Desktop/insta_data')

os.makedirs('/Users/kibeomkim/Desktop/insta_data/test/samsung')
os.chdir('/Users/kibeomkim/Desktop/insta_data/test')
os.makedirs('/Users/kibeomkim/Desktop/insta_data/test/apple')
os.chdir('/Users/kibeomkim/Desktop')

In [4]:
os.listdir('/Users/kibeomkim/Desktop/app_sam')

['apple', '.DS_Store', 'samgalaxy']

In [24]:
app_list = [int(x.split('.')[0]) for x in os.listdir('/Users/kibeomkim/Desktop/app_sam/apple') if x is not '.DS_Store' ]
sam_list = [int(x.split('.')[0]) for x in os.listdir('/Users/kibeomkim/Desktop/app_sam/samgalaxy') if x is not '.DS_Store' ]

In [25]:
import shutil
for image in app_list : 
    if image <= 50 : 
        shutil.copyfile(f'/Users/kibeomkim/Desktop/app_sam/apple/{image}.jpg', f'/Users/kibeomkim/Desktop/insta_data/test/apple/{image}.jpg')
    else : 
        shutil.copyfile(f'/Users/kibeomkim/Desktop/app_sam/apple/{image}.jpg', f'/Users/kibeomkim/Desktop/insta_data/train/apple/{image}.jpg')

for image in sam_list : 
    if image <= 50 : 
        shutil.copyfile(f'/Users/kibeomkim/Desktop/app_sam/samgalaxy/{image}.jpg', f'/Users/kibeomkim/Desktop/insta_data/test/samsung/{image}.jpg')
    else : 
        shutil.copyfile(f'/Users/kibeomkim/Desktop/app_sam/samgalaxy/{image}.jpg', f'/Users/kibeomkim/Desktop/insta_data/train/samsung/{image}.jpg')

In [1]:
# 이미지 데이터 전처리 # 데이터 증식 사용
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/insta_data/train/', 
    classes=['samsung', 'apple'],
    target_size=(150, 150),
    batch_size=50, 
    class_mode='binary', 
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/insta_data/test/',
    classes=['samsung', 'apple'],
    target_size=(150,150),
    batch_size=50,
    class_mode='binary', 
)

Found 1606 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [42]:
from keras import models 
from keras import layers 

# VGG16 합성곱 기반 층 만들기 

from keras.applications import vgg16
from keras import optimizers

# 합성곱 기반 층
conv_base = vgg16.VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150, 150, 3)
)

# 모델 정의
model = models.Sequential() 
model.add(conv_base)# 합성곱 기반 층
model.add(layers.Flatten()) # 합성곱 기반 층 출력 1차원 텐서(벡터로)변형
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# 합성곱 기반 층 가중치 동결 
conv_base.trainable = False
print(f'{len(model.trainable_weights)}') 


# 모델 컴파일 
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.adam_v2.Adam(learning_rate=0.001),
    metrics=['binary_accuracy']
)

# 모델 훈련 
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 32, 
    epochs=50,
    validation_data = validation_generator, 
    validation_steps=2)

8
Epoch 1/50


2022-03-19 10:49:25.893110: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - ETA: 0s - loss: 0.9381 - binary_accuracy: 0.6028

2022-03-19 10:49:40.592309: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 16s 464ms/step - loss: 0.9381 - binary_accuracy: 0.6028 - val_loss: 0.7395 - val_binary_accuracy: 0.5800
Epoch 2/50
32/32 [==============================] - 14s 440ms/step - loss: 0.6865 - binary_accuracy: 0.6754 - val_loss: 0.6307 - val_binary_accuracy: 0.6600
Epoch 3/50
32/32 [==============================] - 14s 442ms/step - loss: 0.5755 - binary_accuracy: 0.7089 - val_loss: 0.6425 - val_binary_accuracy: 0.6400
Epoch 4/50
32/32 [==============================] - 14s 440ms/step - loss: 0.5077 - binary_accuracy: 0.7609 - val_loss: 0.6138 - val_binary_accuracy: 0.6500
Epoch 5/50
32/32 [==============================] - 14s 438ms/step - loss: 0.5055 - binary_accuracy: 0.7526 - val_loss: 0.6430 - val_binary_accuracy: 0.6400
Epoch 6/50
32/32 [==============================] - 14s 438ms/step - loss: 0.4699 - binary_accuracy: 0.7828 - val_loss: 0.6464 - val_binary_accuracy: 0.6500
Epoch 7/50
32/32 [==============================] - 15s 444ms/step - 

KeyboardInterrupt: 

# 미세조정 

In [43]:
conv_base.trainable = True 

set_trainable = False 
for layer in conv_base.layers : 
    if layer.name == 'block5_conv1' : 
        set_trainable = True 
    if set_trainable : 
        layer.trainable = True 
    else : 
        layer.trainable = False 

In [44]:
# 미세조정 

# 컴파일
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.adam_v2.Adam(learning_rate=0.0001),
    metrics=['binary_accuracy']
)
# 훈련
history = model.fit_generator(
    train_generator,
    steps_per_epoch=32, 
    epochs=50, 
    validation_data=validation_generator,
    validation_steps=2,
)

Epoch 1/50


2022-03-19 10:57:36.472861: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - ETA: 0s - loss: 0.3478 - binary_accuracy: 0.8612

2022-03-19 10:57:53.177676: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 18s 536ms/step - loss: 0.3478 - binary_accuracy: 0.8612 - val_loss: 1.5391 - val_binary_accuracy: 0.6200
Epoch 2/50
32/32 [==============================] - 17s 510ms/step - loss: 0.3725 - binary_accuracy: 0.8368 - val_loss: 1.8476 - val_binary_accuracy: 0.6200
Epoch 3/50
32/32 [==============================] - 17s 507ms/step - loss: 0.3196 - binary_accuracy: 0.8708 - val_loss: 1.2124 - val_binary_accuracy: 0.6500
Epoch 4/50
32/32 [==============================] - 17s 507ms/step - loss: 0.3026 - binary_accuracy: 0.8785 - val_loss: 1.4874 - val_binary_accuracy: 0.6000
Epoch 5/50
32/32 [==============================] - 17s 516ms/step - loss: 0.2499 - binary_accuracy: 0.8959 - val_loss: 1.2001 - val_binary_accuracy: 0.6800
Epoch 6/50
32/32 [==============================] - 17s 510ms/step - loss: 0.2770 - binary_accuracy: 0.8798 - val_loss: 1.1763 - val_binary_accuracy: 0.6700
Epoch 7/50
32/32 [==============================] - 17s 506ms/step - 

KeyboardInterrupt: 